In [20]:
from datetime import datetime
start_time0 = datetime.now()
import spacy
import numpy as np
import os
import re
import nltk
import pandas as pd
import got3
import matplotlib.pyplot as plt
import pickle
import xgboost as xgb
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report
import GetOldTweets3 as got #https://github.com/Jefferson-Henrique/GetOldTweets-python

## Etape 1 : construction ML 

In [21]:
train= pd.read_csv('./nlp/tweets.csv', sep='^([^,]+),', engine='python', error_bad_lines=False, encoding='utf-8', index_col=[0])

In [22]:
train.head()

,polarity,statutnull
NaN,0,"- Awww, c'est nul. Tu aurais du prendre David ..."
NaN,0,Est contrarié qu'il ne puisse pas mettre à jou...
NaN,0,J'ai plongé plusieurs fois pour la balle. A ré...
NaN,0,Tout mon corps a des démangeaisons et comme si...
NaN,0,"Non, il ne se comporte pas bien du tout. je su..."


In [23]:
train = train.reset_index(drop=True)

In [24]:
train.head()

,polarity,statutnull
0,0,"- Awww, c'est nul. Tu aurais du prendre David ..."
1,0,Est contrarié qu'il ne puisse pas mettre à jou...
2,0,J'ai plongé plusieurs fois pour la balle. A ré...
3,0,Tout mon corps a des démangeaisons et comme si...
4,0,"Non, il ne se comporte pas bien du tout. je su..."


In [25]:
train.columns

Index(['polarity', 'statutnull'], dtype='object')

In [26]:
train['polarity'] = train['polarity'].str.replace("0","negatif").str.replace("4","positif")

In [27]:
index_zero = train[(train['polarity'] != 'positif') & (train['polarity']!= 'negatif')].index

In [28]:
train.drop(index_zero, inplace=True)

In [29]:
train = train.dropna()

In [30]:
train = train.sample(n=300000)

In [31]:
train['polarity'].value_counts()
#.plot(kind='bar')

positif    150331
negatif    149669
Name: polarity, dtype: int64

In [32]:
def clean_up(tweet):
    tweet = re.sub(r'http\S+|(pic.twitter\.[^\s]+)|(www\.[^\s]+)|(@\S+)|\s\s+|[^\w\s]',' ',tweet) 
    #1:http+suite/2.pic.twitter+suite/3.www.+suite/4.@+suite/5.espaces++/6.ponctuation
    tweet = tweet.lower().strip() #bdc
    tweet = word_tokenize(tweet) #tokenisation
    stop_words = stopwords.words('french') #stopwords nltk
    stop_words.append('rt') #+'rt'
    tweet = [word for word in tweet if word not in stop_words]
    tweet = [word for word in tweet if len(word)>1] #exclus mot de 1 lettre
    return ' '.join(tweet) #retour sans tokenisation, requis par spacy
def stem_spacy(tweet):
    import spacy
    nlp = spacy.load('fr_core_news_sm')
    tweet = tweet.apply(nlp)
    tweet_stem=[]
    for doc in tweet:
        tweet_stem.append([word.lemma_ for word in doc])
    return tweet_stem

In [33]:
train.statutnull = train.statutnull.astype(str)

In [34]:
train['original'] = train['statutnull']

In [35]:
train.statutnull = train.statutnull.apply(lambda s: clean_up(s))

In [36]:
train.head()

,polarity,statutnull,original
1005646,positif,fatigué après tout tennis vais jouer samedi di...,Je suis fatigué après tout ce tennis. Je vais ...
1275286,positif,vient voir disney3d britni amp trop bien pleuré,Vient de le voir dans disney3d w / britni & am...
28070,negatif,essuie glace cool cruise cool,Essuie glace = cool cruise = / = cool
1478002,positif,lol souviens,Lol. Et je me souviens de toi
242289,negatif,cœur brisé tout roger,Le cœur brisé c'est tout ton roger


In [37]:
train.statutnull = stem_spacy(train.statutnull)

In [38]:
train.head()

,polarity,statutnull,original
1005646,positif,"[fatiguer, après, tout, tennis, aller, jouer, ...",Je suis fatigué après tout ce tennis. Je vais ...
1275286,positif,"[venir, voir, disney3d, britni, amp, trop, bie...",Vient de le voir dans disney3d w / britni & am...
28070,negatif,"[essuyer, glacer, cool, cruise, cool]",Essuie glace = cool cruise = / = cool
1478002,positif,"[lol, souvenir]",Lol. Et je me souviens de toi
242289,negatif,"[cœur, briser, tout, roger]",Le cœur brisé c'est tout ton roger


In [39]:
train.rename(columns={'statutnull':'tweet'},inplace=True)

In [40]:
train.shape

(300000, 3)

In [41]:
train.reset_index(drop=True).head()

,polarity,tweet,original
0,positif,"[fatiguer, après, tout, tennis, aller, jouer, ...",Je suis fatigué après tout ce tennis. Je vais ...
1,positif,"[venir, voir, disney3d, britni, amp, trop, bie...",Vient de le voir dans disney3d w / britni & am...
2,negatif,"[essuyer, glacer, cool, cruise, cool]",Essuie glace = cool cruise = / = cool
3,positif,"[lol, souvenir]",Lol. Et je me souviens de toi
4,negatif,"[cœur, briser, tout, roger]",Le cœur brisé c'est tout ton roger


In [42]:
train['tweet_processed2'] = train.tweet.apply(lambda x: ' '.join(x))
train.head()

,polarity,tweet,original,tweet_processed2
1005646,positif,"[fatiguer, après, tout, tennis, aller, jouer, ...",Je suis fatigué après tout ce tennis. Je vais ...,fatiguer après tout tennis aller jouer samedi ...
1275286,positif,"[venir, voir, disney3d, britni, amp, trop, bie...",Vient de le voir dans disney3d w / britni & am...,venir voir disney3d britni amp trop bien pleurer
28070,negatif,"[essuyer, glacer, cool, cruise, cool]",Essuie glace = cool cruise = / = cool,essuyer glacer cool cruise cool
1478002,positif,"[lol, souvenir]",Lol. Et je me souviens de toi,lol souvenir
242289,negatif,"[cœur, briser, tout, roger]",Le cœur brisé c'est tout ton roger,cœur briser tout roger


In [43]:
train_sample = train.sample(n=1000)

In [44]:
count_vectorizer = CountVectorizer(analyzer='word') 
cv = count_vectorizer.fit(train['tweet_processed2'])
pickle.dump(cv, open("cv1.pkl", "wb"))

In [45]:
cv_ = count_vectorizer.fit_transform(train['tweet_processed2'])
cv_.shape

(300000, 986194)

In [46]:
cv1 = pickle.load(open("cv1.pkl", 'rb'))
cv1_new = CountVectorizer(vocabulary = cv1.vocabulary_)
X_cv1 = cv1_new.transform(train_sample['tweet_processed2'])
X_cv1.shape

(1000, 986194)

In [47]:
end_time0 = datetime.now()
print('Temps import et clean: {}'.format(end_time0 - start_time0))

Temps import et clean: 1:21:18.259473


## XGBoost model - non retenu

In [60]:
start_time1 = datetime.now()

In [61]:
X_train_xg,X_test_xg,y_train_xg,y_test_xg = train_test_split(cv_,train['polarity'],test_size=.2,
                                                 stratify=train['polarity'], random_state=42)

In [62]:
xgbc = XGBClassifier(max_depth=9, min_child_weight=4, n_estimators=1000, nthread= 3)
xgbc.fit(X_train_xg,y_train_xg)
prediction_xgb = xgbc.predict(X_test_xg)
print(classification_report(prediction_xgb,y_test_xg))
#0.742425 count vectorizer (max_depth=6, n_estimators=1000, nthread= 3)
#0.7415 tfidf

              precision    recall  f1-score   support

     negatif       0.70      0.78      0.74     26863
     positif       0.80      0.73      0.76     33137

    accuracy                           0.75     60000
   macro avg       0.75      0.75      0.75     60000
weighted avg       0.76      0.75      0.75     60000



In [63]:
print(accuracy_score(prediction_xgb,y_test_xg))

0.752


In [64]:
end_time1 = datetime.now()
print('Temps modèle XGBoost : {}'.format(end_time1 - start_time1))

Temps modèle Naive Bayes: 1:32:10.856343


In [65]:
train_sample['polarity_xgboost']=xgbc.predict(X_cv1)

## Logistic regression model - retenu

In [48]:
start_time2 = datetime.now()

In [63]:
X_train_logreg,X_test_logreg,y_train_logreg,y_test_logreg = train_test_split(cv_,train['polarity'] , test_size=.2,
                                                 stratify=train['polarity'], random_state=42)

In [65]:
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression()
cv_logreg=GridSearchCV(logreg,grid,cv=10)
cv_logreg.fit(X_train_logreg,y_train_logreg)

print("tuned hpyerparameters :(best parameters) ",cv_logreg.best_params_)
print("accuracy :",cv_logreg.best_score_)
#tuned hpyerparameters :(best parameters)  {'C': 0.1, 'penalty': 'l2'}
#accuracy : 0.7391625

C:\Users\aline\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


tuned hpyerparameters :(best parameters)  {'C': 0.1, 'penalty': 'l2'}
accuracy : 0.7620083333333333


In [78]:
logreg = LogisticRegression(C=1,penalty="l2")
logreg.fit(X_train_logreg,y_train_logreg)
prediction_logreg = logreg.predict(X_test_logreg)
print(classification_report(prediction_logreg,y_test_logreg))
#0.745825 count vectorizer
#0.74105 tfidf

              precision    recall  f1-score   support

     negatif       0.74      0.77      0.76     28741
     positif       0.78      0.75      0.77     31259

    accuracy                           0.76     60000
   macro avg       0.76      0.76      0.76     60000
weighted avg       0.76      0.76      0.76     60000



In [62]:
print(accuracy_score(prediction_logreg,y_test_logreg))

0.7628166666666667


In [52]:
#train_sample['polarity_logreg']=logreg.predict(X_cv1)

In [53]:
#accuracy_score(train_sample['polarity_logreg'],train_sample['polarity'])

0.77

In [54]:
end_time2 = datetime.now()
print('Temps modèle Logistic regression: {}'.format(end_time2 - start_time2))

Temps modèle Logistic regression: 0:00:39.488658


## Naive Bayes model - non retenu

In [90]:
train2 = train.sample(100000)

In [27]:
start_time3 = datetime.now()

In [28]:
all_words = []
NUM_FEATURES = 5000
for index, value in train2.tweet.iteritems():
    if value not in all_words:
        all_words += value
top_features = [x[0] for x in nltk.FreqDist(all_words).most_common(NUM_FEATURES)]

In [29]:
df=pd.DataFrame.from_dict(nltk.FreqDist(all_words),orient='index')
df.sort_values(by=0, ascending=False).reset_index().head(10)

,index,0
0,pouvoir,17410
1,aller,17191
2,faire,16256
3,tout,12045
4,plaire,9859
5,ça,9229
6,aimer,8554
7,bon,8500
8,devoir,8496
9,si,8011


In [30]:
def build_features(words):
    features = {}
    for w in top_features:
        features[w] = (w in words)
    return features

In [ ]:
featuresets = []

for index, row in train.iterrows():
    featuresets.append((build_features(row['tweet']), row['polarity']))

In [ ]:
featuresets[:2]

In [ ]:
train_set, test = train_test_split(featuresets, test_size=0.2)
classifier = NaiveBayesClassifier.train(train_set)

In [ ]:
classifier.show_most_informative_features(n=10)

In [ ]:
nltk.classify.accuracy(classifier, test)
# score 73,56% pour sample 100.000 avec stem nltk
# score 73,88% avec sample 100.000 avec stem spacy 

In [ ]:
end_time3 = datetime.now()
print('Temps modèle Naive Bayes: {}'.format(end_time3 - start_time3))

In [ ]:
for i in ranqge(len(train_sample)):
    train_sample['polarity_NB'].iloc[i] = classifier.classify(dict([token,True] for token in train_sample['tweet_processed2'].iloc[i]))

In [ ]:
train_sample=train_sample[['polarity','polarity_xgboost','polarity_logreg','polarity_NB','tweet']]
train_sample

# Etape 2 : twitter query

In [66]:
start_time4 = datetime.now()

In [68]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('#reformedesretraites')\
                                           .setLang("french")\
                                           .setSince("2019-12-05")\
                                           .setUntil("2019-12-31")\
                                           .setMaxTweets(500000)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)

In [69]:
text = [x.text for x in tweet]

In [70]:
date = [x.date for x in tweet]

In [71]:
greve_twitter = pd.DataFrame(columns=['polarity','date','tweet'])

In [72]:
greve_twitter['tweet']=text
greve_twitter['date']=date

In [89]:
greve_twitter.head()

,polarity,date,tweet,tweet_processed,tweet_processed2
0,negatif,2019-12-30 23:58:53+00:00,Comment les travailleurs belges ont bloqué la #RetraiteParPoints #FiersDeLaGreve #reformedesretraites . #GiletsJaunes #greve31decembre #cgt #fo #sudrail #France #Belgique #BFMTV #lci #cnews,comment travailleurs belges bloqué retraiteparpoints fiersdelagreve reformedesretraites giletsjaunes greve31decembre cgt fo sudrail france belgique bfmtv lci cnews,comment travailleur belge bloquer retraiteparpoints fiersdelagreve reformedesretraites giletsjaunes greve31decembre cgt fo sudrail france belgique bfmtv lci cnews
1,negatif,2019-12-30 23:52:46+00:00,#Retraites #ReformeRetraites #reformedesretraites #Macron #GiletsJaunes #greve31decembre #CGT #EdouardPhilippe #SNCF #RATP #FiersDeLaGreve 𝟮𝟳 𝗲𝗺𝗲 𝗷𝗼𝘂𝗿 de #Greve contre la Réforme...,retraites reformeretraites reformedesretraites macron giletsjaunes greve31decembre cgt edouardphilippe sncf ratp fiersdelagreve 𝟮𝟳 𝗲𝗺𝗲 𝗷𝗼𝘂𝗿 greve contre réforme,retraiter reformeretraites reformedesretraites macron giletsjaunes greve31decembre cgt edouardphilippe sncf ratp fiersdelagreve 𝟮𝟳 𝗲𝗺𝗲 𝗷𝗼𝘂𝗿 greve contrer réformer
2,positif,2019-12-30 23:50:37+00:00,#Réformedesretraites : le ton monte entre le ⁦@gouvernementFR⁩ et la #CGT - Le Parisien,réformedesretraites monte entre cgt parisien,réformedesretraites monter entrer cgt parisien
3,negatif,2019-12-30 23:48:15+00:00,Apparemment ça dérange beaucoup ! ! #France #greve31decembre #reformedesretraites #GiletsJaunes #Coulommiers #écologie https://twitter.com/franck77120/status/1211370152933167105,apparemment ça dérange beaucoup france greve31decembre reformedesretraites giletsjaunes coulommiers écologie,apparemment ça déranger beaucoup france greve31decembre reformedesretraites giletsjaunes coulommiers écologie
4,negatif,2019-12-30 23:46:38+00:00,"Tout à fait, un stress permanent qui se rajoutera à celui du travail, de la recherche d'un emploi, de la perte d'un emploi #reformedesretraites #RetraiteParPoints .",tout fait stress permanent rajoutera celui travail recherche emploi perte emploi reformedesretraites retraiteparpoints,tout faire stress permanent rajouter celui travail rechercher emploi perte emploi reformedesretraites retraiteparpoints


In [74]:
greve_twitter['tweet_processed'] = greve_twitter['tweet'].astype(str).apply(lambda s: clean_up(s))

In [75]:
greve_twitter['tweet_processed2'] = stem_spacy(greve_twitter['tweet_processed'])

In [76]:
greve_twitter['tweet_processed2'] = greve_twitter['tweet_processed2'].apply(lambda x: ' '.join(x))
greve_twitter.head()

,polarity,date,tweet,tweet_processed,tweet_processed2
0,NaN,2019-12-30 23:58:53+00:00,Comment les travailleurs belges ont bloqué la ...,comment travailleurs belges bloqué retraitepar...,comment travailleur belge bloquer retraiteparp...
1,NaN,2019-12-30 23:52:46+00:00,#Retraites #ReformeRetraites #reformedesretrai...,retraites reformeretraites reformedesretraites...,retraiter reformeretraites reformedesretraites...
2,NaN,2019-12-30 23:50:37+00:00,#Réformedesretraites : le ton monte entre le ⁦...,réformedesretraites monte entre cgt parisien,réformedesretraites monter entrer cgt parisien
3,NaN,2019-12-30 23:48:15+00:00,Apparemment ça dérange beaucoup ! ! #France #g...,apparemment ça dérange beaucoup france greve31...,apparemment ça déranger beaucoup france greve3...
4,NaN,2019-12-30 23:46:38+00:00,"Tout à fait, un stress permanent qui se rajout...",tout fait stress permanent rajoutera celui tra...,tout faire stress permanent rajouter celui tra...


In [86]:
X_cv2 = cv1_new.transform(greve_twitter['tweet_processed2'])
greve_twitter['polarity']=logreg.predict(X_cv2)

In [87]:
greve_twitter['polarity'].value_counts()

negatif    16606
positif    15669
Name: polarity, dtype: int64

In [80]:
pd.set_option('display.max_colwidth', -1)
greve_twitter_eval = greve_twitter.drop(columns=['tweet_processed','tweet_processed2'])

In [81]:
greve_twitter_eval.sample(10)

,polarity,date,tweet
4508,positif,2019-12-25 11:07:27+00:00,#Macron #GreveGenerale #Greve #greve25decembre #Retraites #Noel #reformedesretraites #LREM #soutienauxgrevistes L'ARGENT MAGIQUE DE L'ELYSÉE... https://twitter.com/EnModeMacaron/status/1209771931727253504
13817,negatif,2019-12-17 17:06:56+00:00,#réformedesretraites Le ras bol de cette réforme. Jusqu au bout!
10433,negatif,2019-12-18 19:23:33+00:00,"Dans les rues il n’y a pas que les grévistes #RATP #SNCF #RER #Bus #Metro Personnels Hospitalier Les Avocats n’en veulent pas de la réforme, les profs non plus Bcp de salariés du privé aimeraient pouvoir être dans la rue aussi #reformedesretraites aussi indigne qu’improductive"
28899,negatif,2019-12-07 09:07:22+00:00,Le dialogue à livre ouvert est la base de la #démocratie et la seule voie de confiance entre le peuple et ses élus. Le #lobbying de l'ombre est une calamité qui mine #InteretGeneral A appliquer de suite par @gouvernementFR pour la #reformedesretraites https://www.lemonde.fr/societe/article/2019/12/04/avant-son-depart-le-president-de-la-hatvp-jean-louis-nadal-appelle-a-renforcer-le-controle-du-lobbying_6021625_3224.html?mediego_ruuid=d741b186-7370-41c8-a91d-e949a18c97ba_8&amp;mediego_euid=492f1c9f21e9510b62cdb3bfb82049e7&amp;mediego_campaign=20191205_fe95ea58-a4c0-4726-a46c-bfaf2274f496&amp;xtor=EPR-32280629-[a-la-une]-20191205-[zone_edito_2_titre_3]
14254,positif,2019-12-17 15:39:56+00:00,Les #pompiers contre la #reformedesretraites #manif17decembre @SnesupFsu @PresseFSU @PompiersParis @PompiersFR @Pompiers_13 @SDIS51 @sdis42
5910,negatif,2019-12-24 05:30:19+00:00,Les syndicats ne pensent qu'à eux mêmes et font croire aux français qu'ils se battent pour eux. Nous ne sommes plus aux temps de Germinal d'Emile où on se battait réellement pour gagner son pain. #majoritésilencieuse #reformedesretraites #ReformeRetraite
16327,negatif,2019-12-17 09:53:03+00:00,Cohérence totale de ce sondage avec celui des #français qui sont pour le retrait de la contre #reformedesretraites https://lepoint.fr/sondages-oui-non/apres-12-jours-de-greve-soutenez-vous-toujours-la-reforme-des-retraites-du-gouvernement-16-12-2019-2353318_1923.php
17871,negatif,2019-12-16 18:53:17+00:00,"Parole d'une infirmière en colère. Et je vous arrête tout de suite, je ne peux faire greve, étant réquisitionnée d'office pour cause de service minimum quotidien. #greve #reformedesretraites #raslebol"
15492,positif,2019-12-17 12:11:22+00:00,Point manifs (police/médias) : Pau : 10 000 Perpignan : 6 500 Avignon : 5 200 St Nazaire : 4 500 Tarbes : 4 000 Besançon : 3 300 Nevers : 3 000 Belfort : 1 800 Mende : 750 Lisieux : 750 Argentan : 400 #greve #grevedu17decembre #reformedesretraites #17decembre2019
10816,negatif,2019-12-18 15:03:25+00:00,"Haut commissaire #reformedesretraites : #Delevoye #Pietraszewski #LaREM a remplacé le cheval de Troie des ""assurances"" par le cheval de Troie de ""l'industrie agro alimentaire"" #Auchan, ancien rapporteur #LoiTravail. #greve18decembre ILS SONT LA CORRUPTION"


In [88]:
greve_twitter.to_csv('greve_twitter_final.csv')

In [83]:
end_time4 = datetime.now()
print('Duration: {}'.format(end_time4 - start_time4))

Duration: 1:17:16.980196
